In [ ]:
!pip install "crewai[tools]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.0/677.0 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 8.4 MB/s 

In [ ]:
# LLM → connects to large language models (OpenAI, Gemini, etc).
# Agent → defines an “AI persona” (with role, goal, backstory).
# Task → defines what each agent needs to do.
# Crew → groups agents + tasks into a workflow.
# SerperDevTool → a tool for web search.

import os
from crewai import LLM, Agent, Task, Crew
from crewai_tools import SerperDevTool

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [ ]:
os.environ["SERPER_API_KEY"] = "your_api" ## For searching the web
os.environ["GEMINI_API_KEY"]="your_api"
os.environ["OPENAI_API_KEY"] = "your_api"
# os.environ["OPENAI_API_KEY"] = os.environ["GEMINI_API_KEY"]
os.environ["OPENAI_API_BASE"] = "https://generativelanguage.googleapis.com/v1beta/openai/"

NameError: name 'os' is not defined

In [ ]:
llm = LLM(
    model = "gemini/gemini-2.0-flash",
    temperature=0.1
)


In [ ]:
llm.call("what is the capital of France?")

'The capital of France is **Paris**.\n'

**Agents**

In [ ]:
# This agent searches the web and collects facts.
research_assistant = Agent(
    role = "Research Assistant Agent",
    goal = "Take the research topic, conduct thorough research, and gather, organize, and summarize relevant information: {topic}",
    backstory = "A meticulous and detail-oriented researcher with strong skills in information gathering, fact-checking, and organizing sources to provide a solid foundation for writing.",
    verbose = True,
    tools = [SerperDevTool()],
    llm = llm
)

In [ ]:
# This agent writes the summary from the facts.
research_writer = Agent(
    role = "Research Writer Agent",
    goal = "Take the organized research and produce a clear, comprehensive, and well-structured research paper or report: {topic}",
    backstory = "An experienced academic writer skilled in synthesizing information into professional, coherent, and well-documented research papers and reports with clarity and precision.",
    verbose = True,
    llm = llm
)

**Tasks**

In [ ]:
task1 = Task(
    description = "Research and compile 3–5 recent and noteworthy facts about {topic} as of 2025.",
    expected_output = "A bulleted list containing 3–5 facts.",
    agent = research_assistant
)

In [ ]:
task2 = Task(
    description = "Using the collected facts, craft a concise 100-word blog-style summary about {topic}.",
    expected_output = "A well-written blog summary (around 100 words).",
    agent = research_writer,
    context = [task1],
)

**Crew**

In [ ]:
crew = Crew(
    agents = [research_assistant, research_writer],
    tasks = [task1, task2],
    verbose = True
)

In [ ]:
crew.kickoff(inputs = {"topic": "The risks of AI"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d1d35358-0fb1-4d20-9e71-dfade9ec0e03                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Assistant Agent                                                                                │
│                                                                                                                 │
│  Task: Research and compile 3–5 recent and noteworthy facts about The risks of AI as of 2025.                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:112: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  self.msg_id = ip.kernel._parent_header['header']['msg_id']

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Writer Agent                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  AI's rapid advancement brings significant risks. Data leaks powered by AI are a top security concern for 69%   │
│  of organizations by 2025, yet 47% lack proper prevention strategies. Alarmingly, 93.2% of organizations        │
│  aren't fully confident in securing AI data. Cisco reports only 4% are "Mature" in AI security readiness. AI    │
│  incidents surged by 56.4% in 2024, totaling 233 reported cases, highlighting growing vulnerabilities.          │
│  Furthermore, AI accelerates misinformation, eroding public trust. Addressing these challenges with robust      │
│  security measures and proactive strategies is crucial to mitigate AI's potential harms and ensure its          │
│  responsible development and deployment.                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4ae1d899-d0b9-4585-80c0-30fefb6c29e2                                                                     │
│  Agent: Research Writer Agent                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d1d35358-0fb1-4d20-9e71-dfade9ec0e03                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: AI's rapid advancement brings significant risks. Data leaks powered by AI are a top security     │
│  concern for 69% of organizations by 2025, yet 47% lack proper prevention strategies. Alarmingly, 93.2% of      │
│  organizations aren't fully confident in securing AI data. Cisco reports only 4% are "Mature" in AI security    │
│  readiness. AI incidents surged by 56.4% in 2024, totaling 233 reported cases, highlighting growing             │
│  vulnerabilities. Furthermore, AI accelerates misinformation, eroding public trust. Addressing these            │
│  challenges with robust security measures and proactive strategies is crucial to mitigate AI's potential harms  │
│  and ensure its responsible development and deployment.                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CrewOutput(raw='AI\'s rapid advancement brings significant risks. Data leaks powered by AI are a top security concern for 69% of organizations by 2025, yet 47% lack proper prevention strategies. Alarmingly, 93.2% of organizations aren\'t fully confident in securing AI data. Cisco reports only 4% are "Mature" in AI security readiness. AI incidents surged by 56.4% in 2024, totaling 233 reported cases, highlighting growing vulnerabilities. Furthermore, AI accelerates misinformation, eroding public trust. Addressing these challenges with robust security measures and proactive strategies is crucial to mitigate AI\'s potential harms and ensure its responsible development and deployment.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Research and compile 3–5 recent and noteworthy facts about The risks of AI as of 2025.', name=None, expected_output='A bulleted list containing 3–5 facts.', summary='Research and compile 3–5 recent and noteworthy facts about The...', raw='